# Tesselation
- Generates tesselation on mouse test set from training 1.

In [1]:
# !pip install patchify

In [2]:
from PIL import Image
import cv2
from PIL import Image
from patchify import patchify, unpatchify
from skimage import data
from skimage import color
from skimage import img_as_float

In [3]:
'Load test set'
data_fld = '../data'
sub_fld = os.path.join(data_fld, 'mouse_data')
tst_path = os.path.join(sub_fld, '0107_pred_script_2.csv')
df_tst = pd.read_csv(tst_path, index_col=0)
df_tst.head(2)

,wsi_id,wsi_name,genotype,x_tile_coord,y_tile_coord,file,batch,animal_id,dissected_region,is_CBTP_old,...,is_test,pred_prob,CBT,CBTA,CBTP,CBT3,CBTPA,CBTP3,prediction,tile_entropy
0,Eran Hodis 2020-09-15-012_Multiple,Eran Hodis 2020-09-15-012.svs,Multiple,28,6,Eran Hodis 2020-09-15-012_28_6_Multiple.png,L41,multiple,TMA01,False,...,True,[9.7655630e-01 3.8493329e-04 7.5743992e-06 2.3...,0.976556,0.000385,7.574399e-06,2.305123e-02,3.337556e-11,1.240714e-12,CBT,0.063171
1,Eran Hodis 2020-09-15-012_Multiple,Eran Hodis 2020-09-15-012.svs,Multiple,40,4,Eran Hodis 2020-09-15-012_40_4_Multiple.png,L41,multiple,TMA01,False,...,True,[3.7247296e-06 1.1470235e-01 4.2321177e-12 1.2...,0.000004,0.114702,4.232118e-12,1.269024e-08,8.852940e-01,1.304739e-10,CBTPA,0.198847


In [4]:
'Create output folder'
ds_fld = os.path.join(home, 'melanoma_images')
ds_fld = os.path.join(ds_fld, '2021_1_7_tessellation_0129')

if os.path.isdir(ds_fld) is False:
    os.mkdir(ds_fld)
classes = ['CBT', 'CBTA', 'CBTP', 'CBT3', 'CBTPA', 'CBTP3']
for cl in classes + ['Multiple']:
    cl_fld = os.path.join(ds_fld, cl)
    if os.path.isdir(cl_fld) is False:
        os.mkdir(cl_fld)

In [5]:
'Load helpers'
colors = {'CBT': 0.1, 'CBT3': 0.2, 'CBTP': 0.3, 'CBTP3': 0.55, 
          'CBTPA': 0.75, 'CBTA': 0.95, 'Uncertain': 0}
saturation = {'CBT': 0.5, 'CBT3': 0.5, 'CBTP': 0.5, 'CBTP3': 0.5, 
              'CBTPA': 0.5, 'CBTA': 0.5, 'Uncertain': 0}
light = {'CBT': 1, 'CBT3': 1, 'CBTP': 1, 'CBTP3': 1, 
         'CBTPA': 1, 'CBTA': 1, 'Uncertain': 0.5}


def colorize(image, hue, saturation=1, light=0):
    """
        Modify hue, saturation and value of an image
    """
    hsv = color.rgb2hsv(image)
    hsv[:, :, 1] = saturation
    hsv[:, :, 0] = hue
    hsv[:, :, 2] = light  # ligt
    return color.hsv2rgb(hsv)

In [6]:
# path to whole slides in PNG format
img_fld = os.path.join(home, '/home/tbiancal/melanoma_dataset/mouse_WSI_PNG')

In [14]:
'Do tesselation'
iterator = df_tst.groupby(['genotype', 'wsi_name',]).count().index.to_frame().values

issues = []
not_found = []

for genotype, wsi_name in tqdm(iterator):
    # load WSI_PNG
    png_name = wsi_name.split('.svs')[0] + '-WSI.png'
    img_path = os.path.join(img_fld, genotype)
    img_path = os.path.join(img_path, png_name)
    wsi_img = cv2.imread(img_path, 3)
    if wsi_img is None:
        not_found.append([genotype, png_name])
    else:
        # Get tile DF for WSI
        mask_1 = df_tst.wsi_name == wsi_name
        mask_2 = df_tst.genotype == genotype
        mask = mask_1 & mask_2
        df_wsi = df_tst[mask]

        # Tessel patches
        patches = patchify(wsi_img, (82, 82, 3), step=82)

        tile_iterator = zip(df_wsi['y_tile_coord'], df_wsi['x_tile_coord'], df_wsi['prediction'])
        for i, j, p in tile_iterator:
            try:
                col_factor = colorize(patches[i, j, 0], colors[p], saturation[p], light[p])  
                patches[i, j, 0] = patches[i, j, 0] * col_factor
            except:
                issues.append([p, i, j, wsi_name, genotype])

        # Path to save the tassellations
        new_img = unpatchify(patches, wsi_img.shape)
        pic = Image.fromarray(new_img)
        out_path = os.path.join(ds_fld, genotype)
        out_path = os.path.join(out_path, os.path.basename(img_path))
        out_pat = out_path.split('.png')[0] + '.pdf'
        pic.save(out_path)

100%|██████████| 98/98 [04:51<00:00,  2.97s/it]


In [9]:
not_found

[['CBT', 'Eran Hodis 2020-09-14-007-WSI.png'],
 ['CBTA', 'EH-15740-022-WSI.png'],
 ['CBTA', 'EH-15740-023-WSI.png'],
 ['CBTP', 'EH-15740-000-WSI.png'],
 ['CBTP', 'EH-15740-001-WSI.png'],
 ['CBTP', 'EH-15740-019-WSI.png'],
 ['CBTP', 'Eran Hodis 2020-09-14-036-WSI.png'],
 ['CBTP', 'Eran Hodis 2020-09-15-026-WSI.png'],
 ['CBTP3', 'EH-15740-028-WSI.png'],
 ['CBTP3', 'EH-15740-030-WSI.png'],
 ['CBTP3', 'Eran Hodis 2020-09-14-019-WSI.png'],
 ['CBTPA', 'Eran Hodis 2020-09-14-013-WSI.png'],
 ['CBTPA', 'Eran Hodis 2020-09-15-013-WSI.png']]

In [13]:
# 'Test on genotype "Multiple"'
# mask = df_tst.genotype == 'Multiple'
# wsi_names = df_tst[mask].wsi_name.unique()

# issues = []
# not_found = []

# for wsi_name in wsi_names:
#     # load WSI_PNG
#     png_name = wsi_name.split('.svs')[0] + '-WSI.png'
#     img_path = os.path.join(img_fld, 'Multiple')
#     img_path = os.path.join(img_path, png_name)
#     assert os.path.isfile(img_path)
#     wsi_img = cv2.imread(img_path, 3)
#     if wsi_img is None:
#         not_found.append([genotype, png_name])
#     else:
#         # Get tile DF for WSI
#         mask_1 = df_tst.wsi_name == wsi_name
#         mask_2 = df_tst.genotype == 'Multiple'
#         mask = mask_1 & mask_2
#         df_wsi = df_tst[mask]

#         # Tessel patches
#         patches = patchify(wsi_img, (82, 82, 3), step=82)

#         tile_iterator = zip(df_wsi['y_tile_coord'], df_wsi['x_tile_coord'], df_wsi['prediction'])
#         for i, j, p in tile_iterator:
#             try:
#                 col_factor = colorize(patches[i, j, 0], colors[p], saturation[p], light[p])  
#                 patches[i, j, 0] = patches[i, j, 0] * col_factor
#             except:
#                 issues.append([p, i, j, wsi_name, genotype])

#         # Path to save the tassellations
#         new_img = unpatchify(patches, wsi_img.shape)
#         pic = Image.fromarray(new_img)
#         out_path = os.path.join(ds_fld, 'Multiple')
#         out_path = os.path.join(out_path, os.path.basename(img_path))
#         pic.save(out_path)